***
# 0. Introduction 
***

# Introduction du README

## Motivations

J'ai découvert personnellement ce framework lors d'un stage de M1 à l'ENS de Lyon, encadré par Louis-Alexandre Couston. Ce stage portait sur l'influence de la forme de l'interface eau-glace sur les propriétés d'un écoulement de Poiseuille. Vous pouvez consulter le rapport de ce stage à ce lien : [Rapport de stage](https://louiscouston.github.io/files/M1SDM_STAGE_Prados_Corentin.pdf).

Pour simuler cet écoulement, j'ai utilisé Dedalus, un outil remarquablement user-friendly qui permet, en peu de temps grâce aux exemples fournis sur le site, de réaliser des simulations d'écoulements en mécanique des fluides. Comme de nombreux utilisateurs de ce framework, je n'ai pas immédiatement cherché à comprendre son fonctionnement interne.

L'objectif principal de ce projet est d'acquérir une compréhension approfondie des bases numériques sous-jacentes à Dedalus, en mettant particulièrement l'accent sur une méthode spécifique appelée la méthode tau. Ce projet vise à m'aider à comprendre le fonctionnement de Dedalus, mais il est également ouvert à toute personne souhaitant découvrir ou approfondir ses connaissances sur le sujet. Je suis naturellement ouvert à toute critique constructive et à tout complément qui serait accueilli avec enthousiasme.

## Références 

Ce projet s'appuie principalement sur le papier suivant :

> K J Burns, G M Vasil, J S Oishi, D Lecoanet, B P Brown, "Dedalus: A Flexible Framework for Numerical Simulations with Spectral Methods," Physical Review Research, vol. 2, no. 2, Apr. 2020.

Ainsi que sur le site et la documentation associée au projet Dedalus :
- Index du site : [https://dedalus-project.org/index.html](https://dedalus-project.org/index.html)
- Documentation : [https://dedalus-project.readthedocs.io/en/latest/index.html](https://dedalus-project.readthedocs.io/en/latest/index.html)

Les documents complémentaires utilisés seront cités lors de leur utilisation.

## Construction du projet 

Les auteurs du projet Dedalus ont opté pour des explications concises, ce qui, de mon point de vue, peut rendre la lecture du papier difficile sans une connaissance préalable du sujet.

Les Jupyter Notebooks rédigés ont donc pour objectif de détailler et d'expliquer chaque point du papier, dans le but de créer une explication pédagogique des choix de la méthode numérique utilisée par les auteurs. Chaque principe sera illustré d'exemples.

Ce travail a été réalisé de manière individuelle, il est donc probablement incomplet et peut contenir des erreurs (y compris dans la compréhension des notions). Les notebooks ont été construits selon le cheminement intellectuel que j'ai suivi pour comprendre les prémices de la méthode tau. La construction des notebooks est progressive, ajoutant peu à peu chaque notion utile pour finalement arriver à la méthode globale utilisée.

Le papier étant riche en informations, ce projet se limite à la partie "SPARSE SPECTRAL METHODS". La partie "Fundamentals of spectral methods" est traité dans le notebook "1_Fondamentaux_sur_les_méthodes_spectrales" et celle nommée "A general sparse tau method" dans le notebook "2_Methode_tau_generale_avec_matrices_creuses". Le notebook "3_Reproduction_figure_papier_Dedalus.ipynb" n'est pas indispensable au projet. Il offre une vision d'ensemble de la partie un et de la partie deux, tout en examinant quelques détails non essentiels.


# Remarque sur la notation du projet

Il convient de souligner que les notations employées dans ce projet ne suivent pas de conventions standard. Ce sont des choix arbitraires qui ont été adoptés au cours du développement du projet.

# Résumé succint du framework

Voici une synthèse de la page [https://dedalus-project.org/about/](https://dedalus-project.org/about/) :

Dedalus est un cadre de résolution d'équations aux dérivées partielles utilisant des méthodes spectrales. Il excelle dans la résolution de problèmes initiaux, aux limites, et d'auto-valeurs généralisées. Parmi ses principales fonctionnalités :

1. **Entrée symbolique des équations :** Dedalus accepte des systèmes presque arbitraires d'équations différentielles et de contraintes algébriques saisis en texte brut. Les termes linéaires sont convertis en systèmes matriciels creux pour des résolutions linéaires efficaces, tandis que les termes non linéaires sont évalués de manière pseudo-spectrale. **La méthode tau (celle qui est au coeur de ce dépôt) est utilisée pour imposer des conditions aux limites presque arbitraires, qui sont également saisies en texte brut.**

2. **Discrétisation dans le domaine spectral :** Dedalus résout sur des domaines discrétisés avec des bases spectrales globales, permettant la construction de domaines cartésiens ou curvilignes.

3. **Parallélisation multidimensionnelle :** Les dimensions séparables des problèmes sont automatiquement parallélisées sur les cœurs disponibles ou sur un maillage de processus spécifié par l'utilisateur, utilisant MPI pour coordonner les calculs.

4. **Discrétisation temporelle implicite-explicite :** Les problèmes de valeurs initiales sont résolus par la discrétisation spectrale de l'espace et l'évolution des coefficients comme un système d'équations différentielles ordinaires couplées. Dedalus propose divers intégrateurs pour la discrétisation temporelle.

5. **Analyse flexible avec HDF5 :** Le format symbolique utilisé pour les équations peut également spécifier des tâches d'analyse, calculées avec les termes non linéaires et enregistrées dans des fichiers HDF5 auto-descriptifs.


# État de l'art

## Les différentes méthodes numériques 
**1. Méthode spectrale : méthode utilisé dans le projet Dedalus**
   1. **Principe :** La méthode spectrale discrétise les variables en les développant dans un ensemble fini de fonctions de base, dérivant ensuite des équations pour les coefficients associés.
   2. **Avantages :** Offre des solutions convergentes rapidement, notamment pour des fonctions lisses.
   3. **Inconvénients :** Limitée à des géométries simples, telles que des boîtes, des cylindres et des sphères.

**2. Autres méthodes :**
   1. **Principe :** Les méthodes traditionnelles (éléments finis, volumes finis, différences finies) discrétisent le domaine des équations aux dérivées partielles (EDP) en cellules ou points.
   2. **Avantages :** Adaptées à des géométries complexes, comme le flux autour d'un avion.
   3. **Inconvénients :** Peuvent être difficiles à implémenter pour des équations complexes, et la convergence est généralement lente avec l'ajout de cellules ou de points.
   

## Pourquoi le projet Dedalus ?

   **1. Différents packages de simulation déjà développés :**
   - **FENICS et FIREDRAKE :** Permettent d'entrer symboliquement des équations en forme variationnelle, produisant des discrétisations par éléments finis adaptées à la modélisation directe et aux calculs d'optimisation. Efficaces pour résoudre un large éventail d'équations aux dérivées partielles (EDP) dans des géométries complexes, mais moins performants que les méthodes spectrales dans des géométries simples.
   - **CHANNELFLOW :** Utilise des méthodes Chebyshev clairsemées pour simuler les équations de Navier-Stokes, mais est limité à résoudre des écoulements incompressibles dans une géométrie de canal périodique.
   - **CHEBFUN et APPROXFUN :** Outils flexibles pour l'approximation de fonctions utilisant des méthodes spectrales. Offrent des fonctionnalités variées, mais ne sont pas optimisés pour la résolution de systèmes d'EDP multidimensionnels sur des architectures parallèles.

**2. Apport du projet Dedalus :**
   - Le projet Dedalus vise à combler les lacunes des solutions déjà existantes en fournissant un cadre appliquant des techniques spectrales modernes et sparses à des simulations fortement parallélisées d'EDP personnalisées. La base de code permet la discrétisation de domaines à l'aide de produits directs de séries spectrales, la spécification symbolique de systèmes d'EDP, et la production automatique d'une discrétisation sparses des équations, tout en parallélisant automatiquement la solution du modèle.

**3. Domaines d'application du projet Dedalus :**
   - Le code Dedalus, open source, modulaire et facile à utiliser, a été développé initialement pour l'étude des écoulements turbulents en astrophysique et géophysique. Cependant, il a été utilisé dans un large éventail de domaines, notamment les mathématiques appliquées, l'astrophysique, les sciences atmosphériques, la biologie, la physique de la matière condensée, la dynamique des fluides, la glaciologie, la limnologie, l'analyse numérique, l'océanographie, la science planétaire et la physique des plasmas.

# Zoom sur le papier associé au projet Dedalus

## Table des matières 
1. INTRODUCTION
2. SPARSE SPECTRAL METHODS
    1. Fundamentals of spectral methods 
        1. Spectral representations of functions 
        2. Common spectral series 
        3. Solving differential equations with spectral methods 
    2. A general sparse tau method 
        1. Sparse differential operators 
        2. Banded boundary conditions 
        3. Nonconstant coefficients 
        4. Solving systems of equations 
3. PROJECT OVERVIEW AND DESIGN 
    1. Codebase structure 
    2. Dependencies 
    3. Documentation 
    4. Community 
4. SPECTRAL BASES 
    1. Fourier basis 
    2. Sine/cosine basis 
    3. Chebyshev basis 
    4. Legendre basis 
    5. Hermite basis 
    6. Laguerre basis 
    7. Compound bases 
    8. Scaled transforms and dealiasing 
    9. Transform plans 
5. DOMAINS 
    1. Parallel data distribution 
    2. Transpose routines 
    3. Distributed data interaction 
6. FIELDS 
    1. Data manipulation 
    2. Field systems 
7. OPERATORS 
    1. Operator classes and evaluation 
    2. Arithmetic operators 
    3. Unary grid operators 
    4. Linear spectral operators 
        1. Differentiation 
        2. Integration 
        3. Interpolation 
        4. Hilbert transforms 
    5. User-specified functions 
    6. Manipulating expressions 
    7. Evaluators 
8. PROBLEMS 
    1. Problem creation 
    2. Variable metadata 
    3. Parameters and nonconstant coefficients 
    4. Substitutions 
    5. Equation parsing 
        1. Linear boundary value problems 
        2. Nonlinear boundary value problems 
        3. Eigenvalue problems 
        4. Initial value problems 
9. SOLVERS 
    1. Matrix construction 
    2. Linear boundary value solver 
    3. Nonlinear boundary value solver 
    4. Eigenvalue solver 
        1. Dense solver 
        2. Sparse solver 
    5. Initial value solver 
        1. Initial conditions 
        2. Time evolution 
        3. Time step determination 
        4. Termination 
    6. Time steppers 
        1. Multistep IMEX integrators 
        2. Runge-Kutta IMEX integrators 
    7. Matrix solvers 
10. ANALYSIS AND POST-PROCESSING 
    1. File handlers 
    2. Analysis tasks 
    3. Post-processing 
11. BENCHMARKS AND EXAMPLES 
    1. Parallel scaling 
    2. Kelvin-Helmholtz accuracy benchmark 
    3. Nonlinear Schrödinger network 
    4. Orszag-Tang vortex 
    5. Quasigeostrophic flow 
    6. Stokes flow 
    7. Atmospheric waves 
    8. Diamagnetic levitation 
12. CONCLUSION AND OUTLOOK 
ACKNOWLEDGMENTS 
APPENDIX: SPECTRAL OPERATOR MATRICES 
1. Differentiation and conversion matrices
    1. Fourier 
    2. Sine/cosine
    3. Chebyshev 
    4. Legendre 
    5. Hermite 
    6. Laguerre 
2. Dirichlet recombination matrices 
    1. Chebyshev and Legendre 
    2. Laguerre
3. NCC multiplication matrices 
    1. Chebyshev 
    2. Legendre 
    3. Hermite 
    4. Laguerre 

## Références 
Ce papier est conséquent et s'appuie sur la référence de plus de 151 articles scientifiques.

## Citations 

À la date du samedi 6 janvier 2024, le projet Dedalus a déjà été cité plus de 339 fois !